In [ ]:
test_url = "https://youtube.com/shorts/a--NSC19MXM?si=0o-IP4UwUZMPmd_6"

In [8]:
import requests

def get_script(url):
    api_endpoint = "http://127.0.0.1:8010/extract_info"
    response = requests.get(api_endpoint, params={"url": url})
    return response.json()

In [9]:
import time
start_time = time.time()
result = get_script("https://youtube.com/shorts/31aKBb7ioZ8?si=BDLcpYFOuqLOYvvc")
print(f"실행시간: {time.time() - start_time} seconds")

실행시간: 2.1851508617401123 seconds


In [11]:
result['script']

[{'text': '여성 리포터 옆으로 딜런 데닉스가 오는데요.', 'start': 0.0, 'end': 2.38},
 {'text': '리포터는 기발한 생각이 떠올랐습니다.', 'start': 2.74, 'end': 5.34},
 {'text': '다른 매니저도 말리지만 좋은 클립이 될 거라면서 계속 해달라고 합니다.',
  'start': 9.96,
  'end': 14.88},
 {'text': '결국에는 들어가는데', 'start': 15.8, 'end': 17.04},
 {'text': '표정이 점점 이상해지다가 그대로 가버리죠.', 'start': 17.04, 'end': 22.56},
 {'text': '여자는 믿을 수 없다는 눈치입니다. 본인도 사실 인지를 못하거든요. 언제 가는지.',
  'start': 30.0,
  'end': 36.06},
 {'text': '리포터는 고맙다며 껴안아줍니다. 어이가 없네요.', 'start': 36.28, 'end': 39.1},
 {'text': '이렇게 경험을 못해봐서 선수들에게 많이 해달라고들 하는데', 'start': 39.5, 'end': 43.06},
 {'text': '머릿속에 기억이 끊기는 느낌이 상당히 이상하죠.', 'start': 43.06, 'end': 46.14}]

In [5]:
import json
import os
if os.path.exists('script.json'):
    os.remove('script.json')

with open('script.json', 'w', encoding='utf-8') as json_file:
    json.dump(result['script'], json_file, ensure_ascii=False, indent=4)

In [1]:
import os
import warnings

from dotenv import load_dotenv
from langchain_teddynote import logging

warnings.filterwarnings(action='ignore')

load_dotenv()  # .env 파일에서 환경 변수를 로드합니다
api_key = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Youtube_Script_Chatbot"
logging.langsmith("Youtube_Script_Chatbot")

LangSmith 추적을 시작합니다.
[프로젝트명]
Youtube_Script_Chatbot


In [2]:
import json
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_teddynote.callbacks import StreamingCallback
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
summary_prompt.template = 'Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. Translate the summary into KOREAN if it is written in ENGLISH.\n5. DO NOT translate any technical terms.\n6. DO NOT include any unnecessary information.\n7. Please refer to each summary and indicate the key topic.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n'
summary_prompt.pretty_print()


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    streaming=True,
    temperature=0.7,
    callbacks=[StreamingCallback()],
)

Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.
7. Please refer to each summary and indicate the key topic.

CONTEXT:
{context}

SUMMARY:"



In [3]:
llm.stream("메롱")

<generator object BaseChatModel.stream at 0x74d89314cbc0>

In [4]:
result = llm.invoke("안녕하세요")

안녕하세요! 어떻게 도와드릴까요?

In [6]:
result.content

'안녕하세요! 어떻게 도와드릴까요?'

In [14]:
from langchain_community.document_loaders import JSONLoader,TextLoader

# JSONLoader로 임시 파일 로드
# jq_schema = '.[] | {content: .text, start_time: .start, end_time: .end}'
jq_schema = ".[].text"
docs = JSONLoader(file_path="script.json", jq_schema=jq_schema,text_content=False).load()

# # 각 Document의 page_content를 디코딩하여 한글로 변환
# for doc in docs:
#     # page_content를 JSON 객체로 변환 후 문자열 디코딩
#     decoded_content = json.loads(doc.page_content) 
#     doc.page_content = decoded_content # JSON 문자열 -> Python dict 변환

In [9]:
docs[0]

Document(metadata={'source': '/home/jinu/my_ws/youtube_script_chatbot/script.json', 'seq_num': 1}, page_content='In the last chapter, you and I started to step through the internal workings of a transformer.')

In [10]:
with open("script.txt","w") as file:
    for text in [doc.page_content for doc in docs]:
        file.write(text+'\n')

In [11]:
docs = TextLoader("script.txt").load()

In [12]:
len(docs[0].page_content)

27899

In [13]:
stuff_chain = create_stuff_documents_chain(llm, summary_prompt)
summary_result = stuff_chain.invoke({"context": docs})

### 주요 주제: Transformer의 내부 작동 원리

- 🚀 **Transformer의 기본 개념**: Transformer는 대형 언어 모델의 핵심 기술로, Attention 메커니즘을 통해 데이터 처리를 시각화함.  
- 📄 **첫 등장**: 이 기술은 2017년의 논문 "Attention is All You Need"에서 처음 소개됨.  
- 🔍 **모델의 목표**: 모델은 주어진 텍스트를 기반으로 다음에 올 단어를 예측하는 것임.  
- 📦 **토큰화**: 입력 텍스트는 토큰으로 분할되며, 이 토큰은 주로 단어 또는 단어의 일부로 구성됨.  
- 🌌 **임베딩**: 각 토큰은 고차원 벡터와 연관되어 있으며, 이 임베딩은 의미를 반영함.  
- ⚖️ **의미의 방향**: 고차원 공간에서 방향이 의미와 대응할 수 있음.  
- 🔄 **Attention 메커니즘**: Attention 메커니즘은 주변 임베딩이 정보를 전달하도록 하여 더 풍부한 의미를 생성함.  
- ✨ **예시**: "mole"이라는 단어는 다양한 맥락에서 다르게 해석되며, Attention은 이를 반영함.  
- 🏰 **단어 임베딩 업데이트**: 예를 들어, "tower"라는 단어는 문맥에 따라 업데이트됨.  
- 🧩 **단일 Attention 헤드**: 단일 Attention 헤드는 하나의 변화량을 계산하여 임베딩을 조정함.  
- 🔑 **쿼리 및 키 매트릭스**: 쿼리와 키 매트릭스는 서로의 관련성을 측정하기 위해 사용됨.  
- 🎲 **소프트맥스**: 점수 분포를 정규화하여 각 단어의 관련성을 계산함.  
- 🎭 **마스킹**: 후속 단어가 이전 단어에 영향을 미치지 않도록 마스킹을 적용함.  
- ⏳ **컨텍스트 크기**: Attention 패턴은 컨텍스트 크기의 제곱에 비례하여 크기가 결정됨.  
- 🧠 **다중 헤드 Attention**: 여러 개의 Attention 헤드를 병렬로 실행하여 다양한 맥락을 학습함.  
- 📊 **모델 파라미터**: GPT-3는 96개의 At

In [17]:
template = """REQUEST:
1. Separate the entire content based on the main topics and group sentences into logical paragraphs.
2. Each paragraph should be structured as a key-value pair in a dictionary. 
3. The keys should be "paragraph1", "paragraph2", and so on. The values should be the corresponding paragraph text.
4. Each sentence in the paragraph should be separated by a newline character.
5. Do not distort the original content under any circumstances.

CONTEXT:
{context}

RESULT:"""

In [18]:
paragraph_separate_template = PromptTemplate(input_variables=['context'],input_types={},partial_variables={},metadata={'lc_hub_owner':"jinu","lc_hub_repo":None,"lc_hub_commit_hash":None},template=template)

In [19]:
stuff_chain = create_stuff_documents_chain(llm, paragraph_separate_template)
paragraph_separate_result = stuff_chain.invoke({"context": docs})

```python
{
    "paragraph1": "In the last chapter, you and I started to step through the internal workings of a transformer.\nThis is one of the key pieces of technology inside large language models, and a lot of other tools in the modern wave of AI.\nIt first hit the scene in a now-famous 2017 paper called Attention is All You Need, and in this chapter, you and I will dig into what this attention mechanism is, visualizing how it processes data.",
    
    "paragraph2": "As a quick recap, here's the important context I want you to have in mind.\nThe goal of the model that you and I are studying is to take in a piece of text and predict what word comes next.\nThe input text is broken up into little pieces that we call tokens, and these are very often words or pieces of words, but just to make the examples in this video easier for you and me to think about, let's simplify by pretending that tokens are always just words.",
    
    "paragraph3": "The first step in a transformer is to ass

In [20]:
import ast
text_data = paragraph_separate_result[paragraph_separate_result.find("{"):paragraph_separate_result.find("}")+1]
parsed_data = ast.literal_eval(text_data)
parsed_data

{'paragraph1': 'In the last chapter, you and I started to step through the internal workings of a transformer.\nThis is one of the key pieces of technology inside large language models, and a lot of other tools in the modern wave of AI.\nIt first hit the scene in a now-famous 2017 paper called Attention is All You Need, and in this chapter, you and I will dig into what this attention mechanism is, visualizing how it processes data.',
 'paragraph2': "As a quick recap, here's the important context I want you to have in mind.\nThe goal of the model that you and I are studying is to take in a piece of text and predict what word comes next.\nThe input text is broken up into little pieces that we call tokens, and these are very often words or pieces of words, but just to make the examples in this video easier for you and me to think about, let's simplify by pretending that tokens are always just words.",
 'paragraph3': 'The first step in a transformer is to associate each token with a high-d

In [21]:
from deep_translator import GoogleTranslator

tl = GoogleTranslator(source='en', target='ko')
translated_data = {}
for k,v in parsed_data.items():
    translated_data[k] = tl.translate(v)

In [22]:
print(translated_data['paragraph1'])

지난 장에서 당신과 저는 트랜스포머의 내부 작동을 살펴보았습니다.
이것은 대규모 언어 모델 내부의 핵심 기술 중 하나이며, 현대 AI의 많은 다른 도구입니다.
이것은 현재 유명해진 2017년 논문인 Attention is All You Need에서 처음 등장했으며, 이 장에서 당신과 저는 이 어텐션 메커니즘이 무엇인지, 그리고 그것이 데이터를 처리하는 방식을 시각화하는 것을 파헤칠 것입니다.


In [19]:
from datetime import datetime

In [23]:
for line in summary_result.strip().split("\n"):
    # 줄이 비어 있지 않다면 Document로 생성하고, 요약 내용임을 metadata로 추가
    if line.strip():
        docs[0].page_content += line+"\n"

In [24]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
split_docs = splitter.split_documents(docs)

In [25]:
embeddings = OpenAIEmbeddings()

In [26]:
vec_store = FAISS.from_documents(split_docs, embeddings)

In [27]:
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

In [28]:
vec_retriever = vec_store.as_retriever( search_kwargs={"k": 10} )

In [29]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[
        bm25_retriever,
        vec_retriever,
    ],
    weights=[0.7, 0.3],
)

In [30]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
chat_prompt = PromptTemplate.from_template(
    """당신은 유튜브 스크립트 기반의 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 영상의 텍스트 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 
만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)


In [50]:
chat_prompt = PromptTemplate.from_template(
"""당신은 유튜브 스크립트 기반의 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
당신의 임무는 주어진 영상의 텍스트 문맥(context)과 내부 지식을 활용하여 주어진 질문(question)에 답하는 것입니다.

1. 검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 
2. 영상과 관련 없는 질문일 경우 "영상과 관계 없는 질문 입니다." 라고 답하세요.
3. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 내부 지식(internal knowledge)을 사용하여 답변을 생성하세요.
4. 만약 문맥에서 답을 찾을 수 없고 내부 지식으로도 답변할 수 없다면, `주어진 정보에서 질문에 대한 답변을 찾을 수 없습니다`라고 답하세요.

한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [51]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [57]:
query = "소프트맥스 함수가 뭔가요?"
ret_docs = ensemble_retriever.invoke(
    query)
for doc in ret_docs:
    print(doc.page_content)
print("-"*100)
response = chain.invoke(query)

이 요약은 Transformer의 기본 작동 원리와 Attention 메커니즘의 중요성을 설명합니다.
the larger dot products, the places where the keys and queries align. For our adjective-noun
that would look a little more like this, where if the keys produced by fluffy and blue really do
align closely with the query produced by creature, then the dot products in these two spots would
would be
some large positive numbers. In the lingo, machine learning people would say that this means the
embeddings of fluffy and blue attend to the embedding of creature. By contrast to the dot
product between the key for some other word like the and the query for creature would be some small
or negative value that reflects that these are unrelated to each other. So we have this grid of
values that can be any real number from negative infinity to infinity giving us a score for how
- 🎲 **소프트맥스**: 점수 분포를 정규화하여 각 단어의 관련성을 계산함.  
- 🎭 **마스킹**: 후속 단어가 이전 단어에 영향을 미치지 않도록 마스킹을 적용함.
- ⚖️ **의미의 방향**: 고차원 공간에서 방향이 의미와 대응할 수 있음.
- 🔑 **쿼리 및 키 매트릭스**: 쿼리와 키 매